In [1]:
"""Testing system changes

Demonstration of how to safely test changes to your data in a production environment

Attributes
----------
instruments
transactions
set holdings
derived portfolios
instrument definitions
"""

'Testing system changes\n\nDemonstration of how to safely test changes to your data in a production environment\n\nAttributes\n----------\ninstruments\ntransactions\nset holdings\nderived portfolios\ninstrument definitions\n'

## The Challenge

You are an asset manager who has recently secured a contract to work with a new small pension fund. The pension fund has decided to switch from their incumbent asset manager on the advice of consultants. The transition is being handled by a transition manager and you have just received the transition accounts with the final confirmed holdings that will make up the initial portfolio for the fund.

As part of the investment agreement with the pension fund they would like to make some changes to the way that they receive their reporting. They would also like to start investing in some exotic over the counter instruments which our internal systems do not support. Therefore in addition to setting up a production portfolio with the initial holdings, you will also need to create a test environment to model these changes without affecting the live environment.

## The Solution

Using LUSID you can mirror your live investment environment into a different segregated scope. This will allow you to continue trading business as usual and driving the best possible performance for your client while simultaneously building and testing our improved reporting and support for more exotic instruments. You can do this by:

1) Creating your live production environment for your new client

2) Creating your instrument universe

3) Adding your take on balances for your client in this live environment

4) Creating your test environment and using derived portfolios to mirror your production environment

5) Making some trades for your client in your production environment and check that they propogate to your test environment

6) Adding some transactions to your test environment and ensuring that they do not affect the live environment

First things first import the libraries that you need and authenticate your LUSID client.

*Run the cell below to import the libraries and authenticate your LUSID client*

In [2]:
# Import LUSID
import lusid
import lusid.models as models
from lusidtools.cocoon.utilities import create_scope_id
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import json
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")
    
print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.6.4789.0


![Initialise](img/simplechanges-lusidinit.gif)

## 1) Create your Live Production Environment

In LUSID you can segregate environments by making use of Scopes. Read more about scopes here [LUSID API Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

Each scope has a unique code which is used to identify it. 

*Run the cell below to create a name for your production scope for your new client*

In [3]:
# Generate a unique id for your scope
client_account_id = create_scope_id()
# Create a name for your scope
production_scope_code = 'production-client-{}'.format(client_account_id)
prettyprint.heading('Production Scope', production_scope_code)

Production Scope: production-client-38a5-07be-4e78-20


![Scopes](img/simplechanges-prodscope.gif)

To manage your client's assets you will need to set up a portfolio for their fund. Read more about portfolios in LUSID here [LUSID API Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

A portfolio lives within a given scope. Each portfolio also has a unique code. This allows you to uniquely identify a portfolio by the combination of its scope and its code. 

*Run the cell below to create a code for the portfolio which will hold your client's fund*

In [4]:
portfolio_code = 'pension-fund-{}'.format(create_scope_id())
prettyprint.heading('Portfolio Code', portfolio_code)

Portfolio Code: pension-fund-38a5-07be-50c1-f6


Now that you have a code for your portfolio you can create it in LUSID. For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that the official transer time from the previous asset manager to you was two days ago. You will therefore need to set the creation date of the portfolio to be two days previous. Read more about the importance of the creation date of a portfolio here [LUSID Knowledge Base: Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [5]:
# Set the official transfer time to be 2 days ago
official_transfer_time = (datetime.now(pytz.UTC) - timedelta(days=2)).isoformat()

# Build our request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name='pension-fund',
    description='Pension fund for new client',
    code=portfolio_code,
    created=official_transfer_time,
    base_currency='GBP',
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=None,
    properties=None)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=production_scope_code,
    create_transaction_portfolio_request=request,
)

# Pretty print the response from LUSID
prettyprint.portfolio_response(response)

Portfolio Created
Scope: production-client-38a5-07be-4e78-20
Code: pension-fund-38a5-07be-50c1-f6
Portfolio Effective From: 2020-07-08 18:18:57.228455+00:00
Portfolio Created On: 2020-07-10 18:18:56.763053+00:00



![Pension Fund Portfolio](img/simplechanges-pensionfund.gif)

## 2) Create your Instrument Universe

To take on your new client's assets you need to ensure that you have all the appropriate instruments loaded into your instrument universe. You can read more about instruments in LUSID here [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument). 

In this case you are going to load the instrumets into LUSID via a CSV that has been prepared for you by the transition manager. In reality you would maintain a comprehensive instrument universe and are likely to already have the instruments that this asset manager holds in your instrument master rendering this step unnecessary. 

*Run the cell below to import your instruents from our the CSV file*

In [6]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv('data/instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,nan
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,nan
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,nan
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,nan
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,nan
5,Glencore_LondonStockEx_GLEN,imd_34534555,GBP,JE00B4T3BW64,BBG001MM1KV4,LN,united_kingdom,GLEN,equity,common_stock,nan
6,JustEat_LondonStockEx_JE,imd_35436366,GBP,GB00BKX5CN86,BBG0065YWM39,LN,united_kingdom,JE/,equity,common_stock,nan
7,Kingfisher_LondonStockEx_KGF,imd_34535552,GBP,GB0033195214,BBG000BKH1W6,LN,united_kingdom,KGF,equity,common_stock,nan
8,MicroFocus_LondonStockEx_MCRO,imd_34567338,GBP,GB00BD8YWM01,BBG000G4KKD2,LN,united_kingdom,MCRO,equity,common_stock,nan
9,RELXGroup_LondonStockEx_REL,imd_43532542,GBP,GB00B2B0DG97,BBG000D03XD4,LN,united_kingdom,REL,equity,common_stock,nan


Now that you have imported the instruments you can upsert them into LUSID. Read more about upsert methods in LUSID here [LUSID Knoweldge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upsert Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [7]:
# Initialise your batch upsert request
batch_upsert_request = {}
# Iterate over your instrument universe
for row, instrument in instrument_universe.iterrows():
    
    # Specify your identifier columns
    identifier_columns = [
        ('isin', 'Isin'), 
        ('figi', 'Figi'), 
        ('ticker', 'Ticker'),
        ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Add the instrument to your batch request
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
                                                                           
# Call LUSID to upsert your batch
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(request_body=batch_upsert_request)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier='Figi')

Instruments Successfully Upserted: 


,Instrument,Figi ID,LUSID Instrument ID
0,UKGiltTreasury_3.5_2045,BBG006N6HZM7,LUID_7D1ZBNYS
1,USTreasury_2.00_2021,BBG00FN3B5K8,LUID_1QZ27FI1
2,UKGiltTreasury_4.5_2034,BBG0000D14P3,LUID_Q7W13KSI
3,Amazon_Nasdaq_AMZN,BBG000BVPXP1,LUID_4SFSCHDX
4,UKGiltTreasury_3.75_2021,BBG001KKJLR4,LUID_WN98IQMI
5,Apple_Nasdaq_AAPL,BBG000B9XVV8,LUID_ITEF0DK5
6,MicroFocus_LondonStockEx_MCRO,BBG000G4KKD2,LUID_4LDYOCLC
7,Kingfisher_LondonStockEx_KGF,BBG000BKH1W6,LUID_VYYXN6C5
8,JustEat_LondonStockEx_JE,BBG0065YWM39,LUID_78DMAZ43
9,Sage_LondonStockEx_SGE,BBG000BN0PP3,LUID_XAJXW68N


![Instrument Master](img/simplechanges-instrumentmaster.gif)

## 3) Add your Take on Balances

Now that you have populated your instrument universe you can take on the holdings that will make up the new client's fund.

The details of these holdings have been sent to you by the transition manager in a CSV file. 

*Run the cell below to import your take on balances from the CSV file*

In [8]:
holdings = pd.read_csv('data/simplechanges-holdings.csv')
holdings.head(n=10)

,instrument_name,quantity,price,figi,client_internal,currency
0,WPP_LondonStockEx_WPP,1050200,8.71,BBG000BF6B57,imd_34536734,GBP
1,UKGiltTreasury_3.5_2045,100674,129.98,BBG006N6HZM7,imd_54234532,GBP
2,UKGiltTreasury_2.0_2025,220456,106.42,BBG0088JSC32,imd_34534536,GBP
3,UKGiltTreasury_4.5_2034,97658,138.47,BBG0000D14P3,imd_34534534,GBP
4,UKGiltTreasury_3.75_2021,320568,108.17,BBG001KKJLR4,imd_34643653,GBP
5,Kingfisher_LondonStockEx_KGF,2879200,2.39,BBG000BKH1W6,imd_34535552,GBP
6,JustEat_LondonStockEx_JE,4789100,5.98,BBG0065YWM39,imd_35436366,GBP
7,RELXGroup_LondonStockEx_REL,6134800,16.57,BBG000D03XD4,imd_43532542,GBP
8,TESCO_LondonStockEx_TSCO,13456500,1.94,BBG000BF46Y8,imd_34634673,GBP
9,Whitebread_LondonStockEx_WTB,8954300,4.57,BBG000BRVH05,imd_35430090,GBP


The official transfer took place 2 days ago. You therefore want to load your holdings into your portfolio to be effective from this date. 

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings)

*Run the cell below to set the holdings on your client's portfolio*

In [9]:
# Initialise your holding adjustments list
holding_adjustments = []

# Create a holding adjustment for each instrument
for index, holding in holdings.iterrows():

    # Set the identifier based on whether the instrument is cash or not
    if 'Cash' in holding['instrument_name']:
        identifier_key = 'Instrument/default/Currency'
        identifer = instrument_name.split('_')[0]
    else:
        identifier_key = 'Instrument/default/Figi'
        identifier = holding['figi']

    # Build your holding adjustment request and append it to your list
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                identifier_key: identifier},
            tax_lots=[
                models.TargetTaxLotRequest(
                    units=holding['quantity'],
                    cost=models.CurrencyAndAmount(
                        amount=holding['quantity'] * holding['price'],
                        currency=holding['currency']),
                    portfolio_cost=holding['quantity'] * holding['price'],
                    price=holding['price'])
            ]
        )
    )
    
# Call LUSID to set the holdings on your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
    scope=production_scope_code,
    code=portfolio_code,
    effective_at=official_transfer_time,
    adjust_holding_request=holding_adjustments)

# Pretty print the response from LUSID
prettyprint.set_holdings_response(response, production_scope_code, portfolio_code)

Holdings Successfully Set for Portfolio
Scope: production-client-38a5-07be-4e78-20
Code: pension-fund-38a5-07be-50c1-f6
Holdings Effective From: 2020-07-08 18:18:57.228455+00:00
Holdings Created On: 2020-07-10 18:18:58.129360+00:00



![Pension Fund Portfolio](img/simplechanges-productionholdings.gif)

## 4) Create your Test Environment

Now that you have created our portfolio in your live trading environment you need to create a test environment to work on building the improved reporting and instrument support that the client has requested. 

To do this you will create a new scope for testing purposes.

*Run the cell below to create a unique code for your testing scope*

In [10]:
test_scope_code = 'test-client-{}'.format(client_account_id)
prettyprint.heading('Test Scope', production_scope_code)

Test Scope: production-client-38a5-07be-4e78-20


![Test Scope](img/simplechanges-testscope.gif)

You can now create a one-way-sync between your portfolio in your production environment and a new derived portfolio in your test environment. Read more about derived portfolios here [LUSID API Knowledge Base: Derived Portfolios](https://support.lusid.com/what-is-a-derived-portfolio-and-how-are-they-used).

For further usage of the create derived portfolio API call refer to the [LUSID API Docs: Create a Derived Portfolio](https://docs.lusid.com/#operation/CreateDerivedPortfolio).

*Run the cell below to create your derived portfolio in the test environment*

In [11]:
# Build your request to create our derived portfolio
request = models.CreateDerivedTransactionPortfolioRequest(
    display_name='pension-fund',
    description='Pension fund for new client',
    code=portfolio_code,
    parent_portfolio_id=models.ResourceId(
        scope=production_scope_code,
        code=portfolio_code),
    created=official_transfer_time)
  
# Call LUSID to create your derived portfolio
response = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi).create_derived_portfolio(
    scope=test_scope_code,
    create_derived_transaction_portfolio_request=request)
    
# Pretty print the response from LUSID
prettyprint.derived_portfolio_response(response)

Portfolio Created
Scope: test-client-38a5-07be-4e78-20
Code: pension-fund-38a5-07be-50c1-f6
Parent Portfolio Scope: production-client-38a5-07be-4e78-20
Parent Portfolio Code: pension-fund-38a5-07be-50c1-f6
Portfolio Effective From: 2020-07-08 18:18:57.228455+00:00
Portfolio Created On: 2020-07-10 18:18:58.716334+00:00



![Derived Portfolio](img/simplechanges-derivedportfolio.gif)

You can check that the derived portfolio has inherited all of the holdings of the parent by retrieving the holdings from each of your portfolios and confirming that they are the same.

For further usage of the get holdings API call refer to the [LUSID API Docs: Get Holdings](https://docs.lusid.com/#operation/GetHoldings).

You can start by getting the holdings from our portfolio in the production scope.

*Run the cell below to get the holdings from your portfolio in the production scope*

In [12]:
# Call LUSID to get your holdings from the portfolio in producton
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=production_scope_code,
    code=portfolio_code,
    effective_at=official_transfer_time,
    property_keys=['Instrument/default/Name'])

# Pretty print the response from LUSID
prettyprint.holdings_response(response, production_scope_code, portfolio_code)

Holdings for Portfolio
Scope: production-client-38a5-07be-4e78-20
Code: pension-fund-38a5-07be-50c1-f6



,Instrument/default/Name,Holding/default/SourcePortfolioId,Holding/default/SourcePortfolioScope,Units,Cost,Currency,Unsettled Transaction Id,Settlement Date
0,WPP_LondonStockEx_WPP,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,1050200.00,9147242.00,GBP,-,-
1,UKGiltTreasury_3.5_2045,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,100674.00,13085203.82,GBP,-,-
2,UKGiltTreasury_2.0_2025,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,220456.00,23461147.98,GBP,-,-
3,UKGiltTreasury_4.5_2034,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,97658.00,13522312.63,GBP,-,-
4,UKGiltTreasury_3.75_2021,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,320568.00,34677443.40,GBP,-,-
5,Kingfisher_LondonStockEx_KGF,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,2879200.00,6892804.80,GBP,-,-
6,JustEat_LondonStockEx_JE,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,4789100.00,28629239.80,GBP,-,-
7,RELXGroup_LondonStockEx_REL,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,6134800.00,101684310.00,GBP,-,-
8,TESCO_LondonStockEx_TSCO,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,13456500.00,26058512.25,GBP,-,-
9,Whitebread_LondonStockEx_WTB,pension-fund-38a5-07be-50c1-f6,production-client-38a5-07be-4e78-20,8954300.00,40912196.70,GBP,-,-


![Derived Portfolio](img/simplechanges-prodholdings.gif)

Now let's retrieve your holdings from the derived portfolio in the test scope.

*Run the cell below to retrieve your holdings from the derived portfolio in the test scope*

In [13]:
# Call LUSID to get your holdings from the portfolio in the test environment
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=test_scope_code,
    code=portfolio_code,
    effective_at=official_transfer_time,
    property_keys=['Instrument/default/Name'])

# Pretty print the response from LUSID
prettyprint.holdings_response(response, test_scope_code, portfolio_code)

Holdings for Portfolio
Scope: test-client-38a5-07be-4e78-20
Code: pension-fund-38a5-07be-50c1-f6



,Instrument/default/Name,Holding/default/SourcePortfolioId,Holding/default/SourcePortfolioScope,Units,Cost,Currency,Unsettled Transaction Id,Settlement Date
0,WPP_LondonStockEx_WPP,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,1050200.00,9147242.00,GBP,-,-
1,UKGiltTreasury_3.5_2045,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,100674.00,13085203.82,GBP,-,-
2,UKGiltTreasury_2.0_2025,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,220456.00,23461147.98,GBP,-,-
3,UKGiltTreasury_4.5_2034,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,97658.00,13522312.63,GBP,-,-
4,UKGiltTreasury_3.75_2021,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,320568.00,34677443.40,GBP,-,-
5,Kingfisher_LondonStockEx_KGF,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,2879200.00,6892804.80,GBP,-,-
6,JustEat_LondonStockEx_JE,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,4789100.00,28629239.80,GBP,-,-
7,RELXGroup_LondonStockEx_REL,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,6134800.00,101684310.00,GBP,-,-
8,TESCO_LondonStockEx_TSCO,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,13456500.00,26058512.25,GBP,-,-
9,Whitebread_LondonStockEx_WTB,pension-fund-38a5-07be-50c1-f6,test-client-38a5-07be-4e78-20,8954300.00,40912196.70,GBP,-,-


![Derived Portfolio](img/simplechanges-testholdings.gif)

## 5) Make some Live Trades in your Production Environment

Even though you have just added the client's portfolio into LUSID you have already made some trades for your new client to ensure that you can outperform their previous manager. You can import these trades from a CSV file. Note that in practice these would ideally come from an order or execution management system. 

*Run the cell below to import our transactions*

In [14]:
transactions = pd.read_csv('data/simplechanges-transactions.csv')
transactions.head()

,transaction_id,instrument_uid,type,units,transaction_price,transaction_currency,figi,client_internal
0,tid_35b77104-7c72-4515-af2b-d22bb45e6edd,WPP_LondonStockEx_WPP,Sell,410000,8.37,GBP,BBG000BF6B57,imd_34536734
1,tid_4f192c6a-e74b-464e-95c2-62a18b092bd8,Kingfisher_LondonStockEx_KGF,Sell,2879200,2.36,GBP,BBG000BKH1W6,imd_34535552
2,tid_3a95473f-1b4a-4652-8047-91624f3f0e19,Glencore_LondonStockEx_GLEN,Buy,669180,2.74,GBP,BBG001MM1KV4,imd_34534555


Now that you have imported your trades from yesterday you can add them to LUSID. Note that for simplicity all trades have the same transaction date and settlement date which you can see in the request below. Read more about transactions in LUSID here [LUSID Knowledge Base: Transactions](https://support.lusid.com/what-is-a-transaction). 

For further usage of the upsert transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://docs.lusid.com/#operation/UpsertTransactions).

*Run the cell below to upsert your trades into LUSID*

In [15]:
# Initialise a list to hold your requests
requests = []

yesterday = (datetime.now(pytz.UTC)-timedelta(days=1)).isoformat()
tomorrow = (datetime.now(pytz.UTC)+timedelta(days=1)).isoformat()

# Iterate over your transactions
for index, transaction in transactions.iterrows():
    
    # Build your request and append it to your list
    requests.append(
        models.TransactionRequest(
            transaction_id=transaction['transaction_id'],
            type=transaction['type'],
            instrument_identifiers={
                'Instrument/default/Figi': transaction['figi']},
            transaction_date=yesterday,
            settlement_date=tomorrow,
            units=transaction['units'],
            transaction_price=models.TransactionPrice(
                price=transaction['transaction_price'],
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=transaction['transaction_price'] * transaction['units'],
                currency=transaction['transaction_currency']),
            source='Client',
            transaction_currency=transaction['transaction_currency']))

# Call LUSID to upsert your transactions
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=production_scope_code,
    code=portfolio_code,
    transaction_request=requests)

# Pretty print the response from LUSID
prettyprint.transactions_response(
    response,
    production_scope_code,
    portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: production-client-38a5-07be-4e78-20
Code: pension-fund-38a5-07be-50c1-f6
Transactions Effective From: 2020-07-09 18:19:00.254020+00:00
Transactions Created On: 2020-07-10 18:18:59.976908+00:00



As you have created a derived portfolio these trades will also appear in your test environment. You can check this by retrieving the transactions from your portfolio in the test environment. 

For further usage of the get transactions API call refer to the [LUSID API Docs: Get Transactions](https://docs.lusid.com/#operation/GetTransactions).

*Run the cell below to get your transactions from the test environment*

In [16]:
# Call LUSID to get all transactions made yesterday
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_transactions(
    scope=test_scope_code,
    code=portfolio_code,
    from_transaction_date=yesterday,
    to_transaction_date=yesterday,
    property_keys=['Instrument/default/Name']
)

# Pretty print the response from LUSID
prettyprint.get_transactions_response(
    response, 
    test_scope_code, 
    portfolio_code, 
    property_keys=['Instrument/default/Name'])

Transactions Retrieved from Portfolio
Scope:  test-client-38a5-07be-4e78-20
Code:  pension-fund-38a5-07be-50c1-f6 



,Transaction ID,Transaction Type,Instrument/default/Name,Units,Price,Currency,Transaction Date
0,tid_35b77104-7c72-4515-af2b-d22bb45e6edd,Sell,WPP_LondonStockEx_WPP,410000.00,8.37,GBP,2020-07-09 18:19:00.254020+00:00
1,tid_4f192c6a-e74b-464e-95c2-62a18b092bd8,Sell,Kingfisher_LondonStockEx_KGF,2879200.00,2.36,GBP,2020-07-09 18:19:00.254020+00:00
2,tid_3a95473f-1b4a-4652-8047-91624f3f0e19,Buy,Glencore_LondonStockEx_GLEN,669180.00,2.74,GBP,2020-07-09 18:19:00.254020+00:00


![Transactions Production Environment](img/simplechanges-transactionsprod.gif)

## 6) Add some Test Transactions in your Test Environment

You now want to add a transaction to your test environment so that your reporting team can test some new functionality on over the counter (OTC) contracts. You will import a overly simplified sample contract from a JSON file. 

*Run the cell below to import your simple OTC swap contract*

In [17]:
# Import our simple over the counter swap contract from a JSON file
simple_otc_swap_contract = open('data/interest_rate_swap.json').read()
simple_otc_swap_contract = json.loads(simple_otc_swap_contract)
# Pretty print our contract
pprint.pprint(simple_otc_swap_contract)

{'InternalContractId': 'irs_85678992',
 'MaturityDate': '04/04/2025',
 'StartDate': '03/04/2015',
 'assetType': 'swap',
 'flow conventions': 'GBP_6m_LIBOR',
 'strike': 0.02234}


Now that you have loaded your contract you need to add it to the instrument master in LUSID. Read more about adding OTC instruments to LUSID here [LUSID Knowledge Base: Creating Bespoke Instruments](https://support.lusid.com/creating-bespoke-instruments).

*Run the cell below to upsert your OTC contract into LUSID*

In [18]:
# Create the definition for your instrument
simple_otc_swap_instrument = models.InstrumentDefinition(
    name=simple_otc_swap_contract['InternalContractId'],
    identifiers={
        'ClientInternal': models.InstrumentIdValue(
            value=simple_otc_swap_contract['InternalContractId'])
    },
    definition=models.ExoticInstrument(
        instrument_format=models.InstrumentDefinitionFormat(
        source_system='ClientSystemA',
        vendor='ClientA',
        version='0.0.1'
        ),
        content=json.dumps(simple_otc_swap_contract),
        instrument_type='ExoticInstrument'            
    ),
    properties=None)

# Call LUSID to upsert your OTC swap contract
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body={'simple_otc_swap_instrument': simple_otc_swap_instrument})

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,simple_otc_swap_instrument,irs_85678992,LUID_D4AY0P56


Now that you have your OTC contract added to the LUSID instrument master you can book a trade against it on your derived portfolio in your test environment.

*Run the cell below to upsert your OTC transaction into the derived portfolio*

In [19]:
in_two_days = (datetime.now(pytz.UTC)+timedelta(days=2)).isoformat()
in_four_days = (datetime.now(pytz.UTC)+timedelta(days=4)).isoformat()

request = models.TransactionRequest(
    transaction_id='tid_35b77104-7c72-4515-af2b-d22bb45e6eff',
    type='Buy',
    instrument_identifiers={
        'Instrument/default/ClientInternal': simple_otc_swap_contract['InternalContractId']},
    transaction_date=in_two_days,
    settlement_date=in_four_days,
    units=1,
    transaction_price=models.TransactionPrice(
        price=0,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=0,
        currency='GBP'),
    source='Client',
    transaction_currency='GBP')

# Call LUSID to upsert your transactions
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=test_scope_code,
    code=portfolio_code,
    transaction_request=[request])

# Pretty print the response from LUSID
prettyprint.transactions_response(response, test_scope_code, portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: test-client-38a5-07be-4e78-20
Code: pension-fund-38a5-07be-50c1-f6
Transactions Effective From: 2020-07-12 18:19:01.399487+00:00
Transactions Created On: 2020-07-10 18:19:01.108053+00:00



Now you can ensure that adding this trade has not affected the production environment. 

*Run the cell below to check to see if any of the transactions can be found in the production environment*

In [20]:
# Call LUSID to see if the transaction from the test environment propogated through
response = api_factory.build(lusid.api.TransactionPortfoliosApi).get_transactions(
    scope=production_scope_code,
    code=portfolio_code,
    from_transaction_date=in_two_days,
    to_transaction_date=in_two_days,
    property_keys=['Instrument/default/Name'])

# Pretty print the response from LUSID
prettyprint.get_transactions_response(
    response, 
    production_scope_code, 
    portfolio_code,
    ['Instrument/default/Name'])

Transactions Retrieved from Portfolio
Scope:  production-client-38a5-07be-4e78-20
Code:  pension-fund-38a5-07be-50c1-f6 



,Transaction ID,Transaction Type,Units,Price,Currency,Transaction Date


![Transactions Production Environment](img/simplechanges-transactionstest.gif)

Now you have both a live environment and a test environment that are linked via a one-way sync. This ensures that any changes to the live environment and prevents you from having to worry about keeping the two in sync. In addition if you need to make any trades to your testing environment you can do this with the assurance that they will not affect your live trading environement. 